# **HOMEWORK 1 - Regressione Lineare**

In questo homework dovrete:

1. Scrivere una funzione di pipeline che deve gestire l' allenamento di un modello di regressione lineare al variare degli iperparametri forniti. Nello specifico:
    * Deve applicare la PCA, se presente.
    
    * Deve applicare la standardizzazione, se presente.

    * Deve applicare la regolarizzazione, se presente.

    * Deve allenare il modello di regressione lineare.

    * Deve calcolare la MAE.

2. Scrivere una funzione che utilizzi la `pipeline` definita al punto 1 e che testi tutte le configurazioni possibili presenti in `configs`. Nel dettaglio la funzione deve:
    * Dividere il dataset in train e validation.

    * Calcolare, grazie alla funzione `pipeline` definita al punto 1, quale configurazione ottiene il punteggio migliore (quale configurazione ha la MAE di validation più bassa).

3. Scrivere una funzione che utilizzi la configurazione migliore prodotta dalla funzione definita al punto 2 e la testi sul test set. 

4. Stampare:
    * La migliore configurazione

    * Il miglior MAE di validation 

    * Il migliore MAE di train

    * Il MAE di test 


Il codice che di seguito trovate già fornito deve essere utilizzato per la risoluzione dell' homework, **NON MODIFICATELO IN ALCUN MODO**.

## **Dataset Wine Quality White**

Il dataset da utilizzare è `wine-quality-white` della libreria `scikit-learn`. Il dataset contiene 11 variabili numeriche + 1 di target che classifica il vino in diverse categorie di qualità. Per il nostro obiettivo la variabile di target è considerata come `float`, permettendoci di applicare la regressione lineare. All' interno del dataset sono contenuti 4898 campioni. 

In [1]:
# Questa cella contiene tutte le librerie di cui necessitate per risolvere l' homework.
# Ricordate di eseguirla prima di iniziare.

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.datasets import fetch_openml
from sklearn.utils import shuffle
from sklearn.preprocessing import StandardScaler

from sklearn.model_selection import train_test_split

In [2]:
hyperparams = {
    # PCA
    'use_pca': [True, False],
    'pca_standardize': [True, False],
    'pca_components': [3, 5, 10],
    # Data standardization
    'data_standardize': [True, False],
    # Regularization l2
    'use_regularization': [True, False],
    'reg_lambda': [0.1, 1, 10],
}

# Calcoliamo tutte le possibili combinazioni di iperparametri
import itertools
combinations = list(itertools.product(*hyperparams.values()))
configs = [dict(zip(hyperparams.keys(), combination)) for combination in combinations]

# Evitiamo le combinazioni non valide
for config in configs:
    if not config['use_pca']:
        config['pca_standardize'] = None
        config['pca_components'] = None
    if not config['use_regularization']:
        config['reg_lambda'] = None
configs = set([tuple(config.items()) for config in configs])

# Convertiamo di nuovo in lista di dizionari
configs = [dict(config) for config in configs]
print(f'Numero di combinazioni: {len(configs)}')

Numero di combinazioni: 56


In [3]:
# Carica il dataset Wine Quality White
data = fetch_openml(name='wine-quality-white', version=1, as_frame=True)
X = data.data
y = data.target.astype(float)  # Assicura che il target sia float per la regressione

def pipeline(X_train, y_train, X_val, y_val, hyperparams):
    """
    Addestra un modello di regressione lineare con eventuale PCA e regolarizzazione L2.
    """
    X_train = X_train.copy()
    X_val = X_val.copy()

    if hyperparams['use_pca']:
        n_components=hyperparams['pca_components']
        if hyperparams['pca_standardize']:       #controllo se occorre standardizzare nella PCA
            scaler_pca = StandardScaler()
            X_train = scaler_pca.fit_transform(X_train)
            X_val = scaler_pca.transform(X_val)

        pca = PCA(n_components)
        X_train = pca.fit_transform(X_train)
        X_val = pca.transform(X_val)

    if hyperparams['data_standardize']:
        # ...
        scaler = StandardScaler()
        X_train = scaler.fit_transform(X_train)
        X_val = scaler.transform(X_val)

    # Aggiunge il termine costante ai dati
    # ...
    vettore_costante_train= np.ones(X_train.shape[0])
    vettore_costante_val= np.ones(X_val.shape[0])
    
    X_train_1= np.c_[X_train , vettore_costante_train]
    X_val_1= np.c_[X_val, vettore_costante_val]
    
    # Calcolo della soluzione di regressione lineare
    if hyperparams['use_regularization']:
        # ...
        reg_lambda= hyperparams['reg_lambda']
        I= np.identity(X_train_1.shape[1])
        parametri = np.linalg.inv(X_train_1.T @ X_train_1 + reg_lambda * I) @ X_train_1.T @ y_train
    
    else:
        # ...
        parametri = np.linalg.inv(X_train_1.T @ X_train_1 ) @ X_train_1.T @ y_train
    

    # Calcolo predizioni
    # ...
    y_train_predizioni = X_train_1 @ parametri
    y_val_predizioni = X_val_1 @ parametri

    
    # Calcola il MAE
    # ...
    mae_train = np.mean(np.abs(y_train - y_train_predizioni))
    mae_val = np.mean(np.abs(y_val - y_val_predizioni))
    
    #return ...
    return mae_train, mae_val

# Dividi il dataset in training e test set
# ...
lunghezza = len(X)
                            
n_train = int(0.8 * lunghezza)                # Suddivisione: 80% training, 20% test
X_train= X[:n_train]
y_train = y[:n_train]
X_test = X[n_train:]
y_test = y[n_train:]

# Dividi il training set in training set effettivo e validation set
# ...
n_train_finale = int(0.75 * len(X_train))     #Suddivido il training set in training e validation (75% - 25% del training)
X_train_effettivo = X_train[:n_train_finale]
y_train_effettivo = y_train[:n_train_finale]
X_val = X_train[n_train_finale:]
y_val = y_train[n_train_finale:]

# Trova la configurazione di iperparametri migliore
# ...
best_config = None
best_val_mae = float('inf')  # inizializzo con un valore molto grande
best_train_mae = None

for config in configs:
    X_train_effettivo_copia = X_train_effettivo.copy() 
    X_val_copia = X_val.copy() 
    train_mae, val_mae = pipeline(X_train_effettivo_copia, y_train_effettivo, X_val_copia, y_val, config)
    
    if val_mae < best_val_mae:
        best_val_mae = val_mae
        best_train_mae = train_mae
        best_config = config
        
# Riallena il modello sul training set completo
# ...
train_mae, test_mae = pipeline(X_train, y_train, X_test, y_test, best_config)

# Calcola il MAE sul test set

# Ho calcolato il MAE sul test set con la pipeline
 

# Stampa  risultati
# ...
print("Migliore configurazione di iperparametri:")
print(best_config)
print(f"MAE migliore sul validation set: {best_val_mae:.4f}")
print(f"MAE sul training set (con la miglior config): {best_train_mae:.4f}")
print(f"MAE sul test set (dopo il riallenamento): {test_mae:.4f}")

/usr/local/lib/python3.11/dist-packages/sklearn/datasets/_openml.py:968: FutureWarning: The default value of `parser` will change from `'liac-arff'` to `'auto'` in 1.4. You can set `parser='auto'` to silence this warning. Therefore, an `ImportError` will be raised from 1.4 if the dataset is dense and pandas is not installed. Note that the pandas parser may return different data types. See the Notes Section in fetch_openml's API doc for details.
  warn(


Migliore configurazione di iperparametri:
{'use_pca': False, 'pca_standardize': None, 'pca_components': None, 'data_standardize': True, 'use_regularization': True, 'reg_lambda': 10}
MAE migliore sul validation set: 0.5873
MAE sul training set (con la miglior config): 0.5975
MAE sul test set (dopo il riallenamento): 0.5484


In `configs` avete una lista di dizionari, ogni dizionario contiene una possibile combinazione di hyperparametri da utilizzare nella fase di training. 